In [2]:
import numpy as np
import time
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.constants import LogicFamily 
# TWO_POINT_FIVE_V, THREE_POINT_THREE_V, FIVE_V

from nidaqmx import stream_readers 

import matplotlib.pyplot as plt

import numpy as np

In [3]:
class DataAcquisition(object):
    # Currently, this class only handles configuration and reading of a single AI channel

    def __init__(self, 
        ch_ai, ch_clock, ch_trig, 
        # clock_rate, sampling_rate, 
        sampling_rate, 
    ):
        self.ch_ai = ch_ai
        self.ch_clock = ch_clock
        self.ch_trig = ch_trig
        # self.clock_rate = clock_rate
        self.sampling_rate = sampling_rate
        self.clock_edge = Edge.RISING
        self.trig_edge = Edge.RISING

        # allocate the read tasks and readers
        self.read_tasks = []
        self.readers = []
        self.buffers = [] 


    def config_readtask(self, n_samples, min_volt=-10.0, max_volt=10.0):
        try:
            # Create and configure an analog input voltage task
            task = nidaqmx.Task()
            channel = task.ai_channels.add_ai_voltage_chan(
                self.ch_ai,"",
                # TerminalConfiguration.RSE,
                TerminalConfiguration.DIFF,
                min_volt,max_volt,
                VoltageUnits.VOLTS
            )
            # Configure sample clock
            task.timing.cfg_samp_clk_timing(
                self.sampling_rate,
                self.ch_clock,
                self.clock_edge,
                AcquisitionType.FINITE, 
                n_samples)
            # read_task.timing.ai_conv_src = self.ch_clock
            # read_task.timing.ai_conv_active_edge = self.clock_edge

            # Configure start trigger
            read_trig = task.triggers.start_trigger
            read_trig.cfg_dig_edge_start_trig(self.ch_trig,self.trig_edge)

            # Configure reader stream
            reader = stream_readers.AnalogSingleChannelReader(task.in_stream)
            # reader = stream_readers.AnalogMultiChannelReader(task.in_stream)
            reader.read_all_avail_samp  = True
            self.read_tasks.append(task)
            self.readers.append(reader)
            buffer = np.zeros(n_samples, dtype=np.float64, order='C')
            self.buffers.append(buffer)
        except Exception as excpt:
            print("fuck you")
            print(excpt)
            # logger.info(f'Error configuring DAQ. Please check your DAQ is connected and powered. Exception details: {type(excpt).__name__} {excpt}')
            self.clear_preallo_all()
        return task

    def read(self, idx=0, timeout=10):
        num_read = self.readers[idx].read_many_sample(
                self.buffers[idx],
                READ_ALL_AVAILABLE,
                timeout
            )
        return self.buffers[idx]

    def clear_preallo(self, idx=0):
        del self.read_tasks[idx]
        del self.read_tasks[idx]
        del self.read_tasks[idx]

    def clear_preallo_all(self):
        self.read_tasks = []
        self.readers = []
        self.buffers = [] 

    def close_readtask(self, idx=0):
        self.read_tasks[idx].close()
        self.clear_preallo(idx)

    def close_readtask_all(self):
        for tk in self.read_tasks:
            tk.close()
        self.clear_preallo_all()

    

In [62]:
DAQch_APD = "/Dev1/ai16"
DAQch_Clock = "/Dev1/PFI8" # clock source
DAQch_Trig = "/Dev1/PFI9" # clock gate


ch_ai = DAQch_APD
ch_clock = DAQch_Clock
ch_trig = DAQch_Trig
# clock_rate = 10E6 # 10MHz.max ext clock rate of NI6343
sampling_rate = 0.5E6 # 500kHz .max ext clock rate of NI6343
daq = DataAcquisition(ch_ai, ch_clock, ch_trig, sampling_rate)

In [64]:
n_samples = 1000
daq.close_readtask_all()
daq.config_readtask(n_samples, min_volt=-10.0E-3, max_volt=10.0E-3)


Task(name=_unnamedTask<1>)

In [69]:
num_record = 1
volt_array = np.zeros(num_record)
time_array = np.zeros(num_record)
dt = n_samples/sampling_rate
for ii in range(num_record):
    dataread = daq.read()
    volt_array[ii] = np.mean(dataread)
    time_array[ii] = ii*dt
    # print(f"length: {len(dataread)}")


In [68]:
len(dataread)

1000

In [ ]:
plt.plot(time_array, volt_array)
plt.savefig("sdgdsgf.png")

In [ ]:

from nspyre import InstrumentGateway

# # connect to the instrument gateway
gw = InstrumentGateway()
gw.connect()

In [ ]:
max_laser_power = gw.laser.get_max_laser_power()
max_laser_current = gw.laser.get_max_laser_current()

In [ ]:
adictlist = [{"value":523, "disabled":True}, {"value":32432, "disabled":True}, {"value":5325, "disabled":True}]
for ino in adictlist:
    ino["disabled"] = False

In [11]:

task = nidaqmx.Task(new_task_name="PL Trace")
DAQch_APD = "/Dev1/ai16"
min_volt = -5
max_volt = 5
n_samples = 5000
refresh_rate = 30.0 # [Hz]
sampling_rate = n_samples*refresh_rate
print(f"sampling rate: {sampling_rate}")
# sampling_rate = 1E2

ch_clock = ""
clock_edge = Edge.RISING


timeout = 10
channel = task.ai_channels.add_ai_voltage_chan(
    DAQch_APD,"",
    # TerminalConfiguration.RSE,
    TerminalConfiguration.DIFF,
    min_volt, max_volt,
    VoltageUnits.VOLTS
)
# Configure sample clock
task.timing.cfg_samp_clk_timing(
    sampling_rate,
    ch_clock,
    clock_edge,
    AcquisitionType.FINITE, 
    n_samples)
# read_task.timing.ai_conv_src = self.ch_clock
# read_task.timing.ai_conv_active_edge = self.clock_edge

# Configure reader stream
reader = stream_readers.AnalogSingleChannelReader(task.in_stream)
# reader = stream_readers.AnalogMultiChannelReader(task.in_stream)
reader.read_all_avail_samp  = True


sampling rate: 150000.0


In [12]:
# task.start()
buffer = np.zeros(n_samples, dtype=np.float64, order='C')
num_trace = 100
data_array = np.zeros(num_trace, dtype=np.float64, order='C')
for ii in range(num_trace):
    time_start = time.time()
    num_read = reader.read_many_sample(
            buffer,
            READ_ALL_AVAILABLE,
            timeout
        )
    data_array[ii] = np.mean(buffer)
    time_ran = time.time()-time_start
    time.sleep(max(1/refresh_rate-time_ran, 0))

print(len(buffer))
print(buffer[50])

5000
-0.0014208073350577832


In [14]:
task.stop()

In [120]:
task.close()

In [4]:
a = np.array([5,6,7])

In [6]:
np.copy(a)

array([5, 6, 7])

In [10]:
np.zeros(56, dtype=np.float64, order='C')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])